In [1]:
%load_ext autoreload
%autoreload 2
import os, sys, re, datetime, random, gzip, json, copy
from pathlib import Path
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import collections
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from scipy import spatial
from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
PROJ_PATH = Path(os.path.join(re.sub("/CTGCN.*$", '', os.getcwd()), 'CTGCN'))
sys.path.insert(1, str(str(PROJ_PATH.parents[0] / 'DySubG/src/')))
from ranking import Evaluation
# exec(open(str(PROJ_PATH.parents[0] / 'DySubG/src/ranking.py')).read())

## Co-authorship

In [4]:
def read_node_embedding(exp, method, num_time_steps):
    node_embedding = {}
    nodes = pd.read_csv(f'./data/{exp}/nodes_set/nodes.csv', names=['nodes'])['nodes'].values
    for time_id in range(num_time_steps-2, num_time_steps):
        embs = pd.read_csv(
            './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t').values
        dict_embs = {k:v for k,v in zip(nodes, embs)}
        node_embedding[time_id] = dict_embs
    return node_embedding

def make_prediction(author_idx, authors, node_embedding, num_time_steps=8):
    time_id = num_time_steps - 2
    k = 50
    pred = []
    for i, aid in enumerate(author_idx):
#         if i % 10 == 0: print(i)
        distances = []
        author_embed = node_embedding[time_id][aid]
        for co in authors:
            if co != aid:
                co_author_embed = node_embedding[time_id][co]
                dist = spatial.distance.cosine(author_embed, co_author_embed)
                distances.append((aid, co, dist))
        distances.sort(key=lambda tup: tup[2])
        pred += distances[:k]
    pd_pred = pd.DataFrame(pred, columns=['author', 'co_author', 'sims'])
    return pd_pred

def eval_ranking(pred_dict, true_dict, k):
    predicted_indices = [] # collections.OrderedDict()
    true_indices = [] # collections.OrderedDict()
    author_indices = sorted(list(true_dict.keys()))
    for aid in author_indices:
        predicted_indices.append(pred_dict[aid])
        true_indices.append(true_dict[aid])
    eval_agent = Evaluation(predicted_indices, true_indices, k)
    return eval_agent.result

def print_report(exp='dblp', k=20, num_time_steps=8, recompute=True, methods=[], selected_methods=[]):
    if len(methods) == 0:
        methods = [
            'GCN', 'GAT', 'SAGE', 'GIN', 
            'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
            'TIMERS', 'DynAE', 'DynRNN', 'DynAERNN', 'DynGEM', 
            'VGRNN', 'EvolveGCN', 'CTGCN-C',
        ]
    if len(selected_methods) == 0:
        selected_methods = [
            'GCN', 'TgGAT', 'TgSAGE', 'TgGIN', 'DynAE', 
            'DynRNN', 'DynAERNN', 'DynGEM', 'EvolveGCN', 
            'VGRNN', 'CTGCN-C']
        
    try:
        pd_edges = pd.read_csv(f'./data/{exp}/0.input/temporal_edge_list.txt', sep=' ', names=['source_id', 'target_id', 'time_id'])
    except:
        pd_edges = pd.read_csv(f'./data/{exp}/0.input/edges.csv')
    
#     data = pd.read_pickle(f'./data/{exp}/0.input/data.pkl')
    node_types = pd.read_csv(f'../DySubG/dataset/dblp/node_types.csv')
    authors = sorted(node_types[node_types['node_type_name']=='author']['node_id'].values)
    
#     num_time_steps = pd_edges['time_id'].max() + 1
    ground_truth = pd.read_pickle(f'./data/{exp}/0.input/ground_truth.pkl')
    author_idx = sorted(ground_truth.keys())
    
    res = []
    for method in methods:
        print(method)
        if os.path.exists(f'./data/{exp}/0.input/{method}_pred.csv') and not recompute:
            pd_pred = pd.read_csv(f'./data/{exp}/0.input/{method}_pred.csv')
        else:
            node_embedding = read_node_embedding(exp, method, num_time_steps)
            pd_pred = make_prediction(author_idx, authors, node_embedding, num_time_steps)
            pd_pred.to_csv(f'./data/{exp}/0.input/{method}_pred.csv', index=False)
        
        pred = pd_pred.groupby('author').agg({'co_author': list}).to_dict()['co_author']
        results = eval_ranking(pred, ground_truth, k)
        tmp = results[results['k']==k]
        tmp['method'] = method
        res.append(tmp[['method', 'k', 'recall', 'mrr', 'map', 'ndcg']])
    
    df = pd.concat(res)

    print('Full report')
    display(df)

    print('Selected methods')
    display(df[df['method'].isin(selected_methods)])
    return df

In [17]:
methods = [
    'GCN', 'GAT',
    'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
    'TIMERS', 'DynAE', #'DynRNN', 
    'DynAERNN', 'DynGEM', 
    'VGRNN',
    'EvolveGCN', 'CTGCN-C',]

report = print_report('dblp', k=10, methods=methods, recompute=False)

GCN


/home/hoang/miniconda2/envs/pygnn/lib/python3.7/site-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


GAT
TgGCN
TgGAT
TgSAGE
TgGIN
TIMERS
DynAE
DynAERNN
DynGEM
VGRNN
EvolveGCN
CTGCN-C
Full report


,method,k,recall,mrr,map,ndcg
10,GCN,10,0.354715,0.501513,0.301855,0.363781
10,GAT,10,0.047845,0.053498,0.024352,0.036737
10,TgGCN,10,0.173360,0.241700,0.128495,0.165419
10,TgGAT,10,0.222234,0.296272,0.162420,0.208323
10,TgSAGE,10,0.077817,0.089820,0.047130,0.065000
10,TgGIN,10,0.177133,0.277165,0.145333,0.181739
10,TIMERS,10,0.279478,0.385739,0.222857,0.276352
10,DynAE,10,0.161054,0.239675,0.135989,0.164583
10,DynAERNN,10,0.081166,0.117677,0.073443,0.084777
10,DynGEM,10,0.139842,0.208327,0.116578,0.142428


Selected methods


,method,k,recall,mrr,map,ndcg
10,GCN,10,0.354715,0.501513,0.301855,0.363781
10,TgGAT,10,0.222234,0.296272,0.162420,0.208323
10,TgSAGE,10,0.077817,0.089820,0.047130,0.065000
10,TgGIN,10,0.177133,0.277165,0.145333,0.181739
10,DynAE,10,0.161054,0.239675,0.135989,0.164583
10,DynAERNN,10,0.081166,0.117677,0.073443,0.084777
10,DynGEM,10,0.139842,0.208327,0.116578,0.142428
10,VGRNN,10,0.106373,0.158931,0.074483,0.101015
10,EvolveGCN,10,0.121292,0.175126,0.098993,0.120770
10,CTGCN-C,10,0.094484,0.120076,0.061203,0.082084


In [21]:
exp = 'dblp'
method = 'GCN'
import torch
for time_id in range(8):
    embs = pd.read_csv(
            './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t')
    torch.save(torch.tensor(embs.values), './{}_gin_gcn_embeddings.pth'.format(time_id))

In [20]:
torch.tensor(embs.values)

tensor([[ 0.0022,  0.0028, -0.0096,  ...,  0.0029,  0.0004, -0.0132],
        [ 0.0094,  0.0003, -0.0032,  ..., -0.0003, -0.0043, -0.0025],
        [ 0.0017,  0.0042, -0.0103,  ...,  0.0025, -0.0011, -0.0083],
        ...,
        [ 0.0035,  0.0018, -0.0142,  ...,  0.0008,  0.0008, -0.0098],
        [-0.0042, -0.0002,  0.0048,  ..., -0.0023, -0.0028, -0.0022],
        [-0.0012, -0.0016, -0.0010,  ...,  0.0020,  0.0021,  0.0003]],
       dtype=torch.float64)

## Expertise

In [2]:
def get_training_data(data, cname2cid, node_embedding):
    all_labels = list(cname2cid.keys())
    train_pos = []
    train_neg = []
    val_pos = []
    val_neg = []
    test_pos = []
    test_neg = []
    
    for i,d in data.items():
        node_id = d['node_id']
        time_id = d['time_id']
        pos_labels = d['label']
        if 'neg_label' in d:
            neg_labels = d['neg_label']
        else:
            neg_labels = [i for i in all_labels if i not in pos_labels]
        if len(pos_labels) > 0:
            for l in pos_labels:
                label_nid = cname2cid[l]
                pos_pair = (
                    np.array(node_embedding[time_id][node_id]), 
                    np.array(node_embedding[time_id][label_nid]))
                if d['dataset'] == 'train':
                    train_pos.append(pos_pair)
                elif d['dataset'] == 'val':
                    val_pos.append(pos_pair)
                elif d['dataset'] == 'test':
                    test_pos.append(pos_pair)
        if len(neg_labels) > 0:   
            for l in neg_labels:
                label_nid = cname2cid[l]
                neg_pair = (
                    np.array(node_embedding[time_id][node_id]), 
                    np.array(node_embedding[time_id][label_nid]))
                if d['dataset'] == 'train':
                    train_neg.append(neg_pair)
                elif d['dataset'] == 'val':
                    val_neg.append(neg_pair)
                elif d['dataset'] == 'test':
                    test_neg.append(neg_pair)
    return train_pos, train_neg, val_pos, val_neg, test_pos, test_neg


def get_link_score(fu, fv, operator='HAD'):
    """Given a pair of embeddings, compute link feature based on operator (such as Hadammad product, etc.)"""
    fu = np.array(fu)
    fv = np.array(fv)
    if operator == 'HAD':
        return np.multiply(fu, fv)
    elif operator == 'AVG':
        return (fu + fv) / 2
    elif operator == 'L1':
        return np.abs(fu - fv)
    elif operator == 'L2':
        return (fu - fv) ** 2
    else:
        raise NotImplementedError

def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
def predict_link_without_classifier(train_pos, train_neg, val_pos, val_neg, test_pos, test_neg):
    pred_train = [sigmoid(np.dot(e[0], e[1].T)) for e in train_pos + train_neg]
    label_train = [1] * len(train_pos) + [0] * len(train_neg)
    pred_val = [sigmoid(np.dot(e[0], e[1].T)) for e in val_pos + val_neg]
    label_val = [1] * len(val_pos) + [0] * len(val_neg)
    pred_test = [sigmoid(np.dot(e[0], e[1].T)) for e in test_pos + test_neg]
    label_test = [1] * len(test_pos) + [0] * len(test_neg)
    return pred_train, label_train, pred_val, label_val, pred_test, label_test
    
def predict_link_with_classifier(train_pos, train_neg, val_pos, val_neg, test_pos, test_neg, operator):
    train_feats = np.array([get_link_score(e[0], e[1], operator) for e in train_pos + train_neg])
    val_feats = np.array([get_link_score(e[0], e[1], operator) for e in val_pos + val_neg])
    test_feats = np.array([get_link_score(e[0], e[1], operator) for e in test_pos + test_neg])
    label_train = np.array([1] * len(train_pos) + [0] * len(train_neg))
    label_val = np.array([1] * len(val_pos) + [0] * len(val_neg))
    label_test = np.array([1] * len(test_pos) + [0] * len(test_neg))
    
    clf = linear_model.LogisticRegression(max_iter=5000)
#     clf.fit(train_feats, label_train)
    clf.fit(np.concatenate((train_feats, val_feats)), 
            np.concatenate((label_train, label_val)))
    pred_train = clf.predict_proba(train_feats)[:, 1]
    pred_val = clf.predict_proba(val_feats)[:, 1]
    pred_test = clf.predict_proba(test_feats)[:, 1]

    return pred_train, label_train, pred_val, label_val, pred_test, label_test, clf

def evaluate_classifier(train_pos, train_neg, val_pos, val_neg, test_pos, test_neg, operators=['HAD'], threshold=0.5):
    results = {}
    models = {}
    # Predict without classifier
    pred_train, label_train, pred_val, label_val, pred_test, label_test = predict_link_without_classifier(
        train_pos, train_neg, val_pos, val_neg, test_pos, test_neg)
    results['sigmoid'] = {
        'train': roc_auc_score(label_train, pred_train), 
        'val': roc_auc_score(label_val, pred_val),
        'test': roc_auc_score(label_test, pred_test), 
    }
    results['sigmoid_f1']  = {
        'train': f1_score(label_train, [1 if i >= threshold else 0 for i in pred_train]), 
        'val': f1_score(label_val, [1 if i >= threshold else 0 for i in pred_val]),
        'test': f1_score(label_test,[1 if i >= threshold else 0 for i in pred_test]),
    }
    
    # Predict with classifier
    for operator in operators:
        pred_train, label_train, pred_val, label_val, pred_test, label_test, clf = predict_link_with_classifier(
            train_pos, train_neg, val_pos, val_neg, test_pos, test_neg, operator)
        results[operator] = {
            'train': roc_auc_score(label_train, pred_train), 
            'val': roc_auc_score(label_val, pred_val),
            'test': roc_auc_score(label_test, pred_test), 
        }
        results[f'{operator}_f1']  = {
            'train': f1_score(label_train, [1 if i >= threshold else 0 for i in pred_train]), 
            'val': f1_score(label_val, [1 if i >= threshold else 0 for i in pred_val]),
            'test': f1_score(label_test,[1 if i >= threshold else 0 for i in pred_test]),
        }
        models[operator] = clf
    return results, models

def eval_lp(data, cname2cid, method='CTGCN-C', num_time_steps=8, exp='dblp', operators=['HAD'], verbose=True, threshold=0.5):
    node_embedding = {}
    nodes = pd.read_csv(f'./data/{exp}/nodes_set/nodes.csv', names=['nodes'])['nodes'].values
    start_idx = min([d['time_id'] for i,d in data.items()])
    for time_id in range(start_idx, num_time_steps):
        embs = pd.read_csv(
            './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t').values
        dict_embs = {k:v for k,v in zip(nodes, embs)}
        node_embedding[time_id] = dict_embs
    train_pos, train_neg, val_pos, val_neg, test_pos, test_neg = get_training_data(
        data, cname2cid, node_embedding)
    if verbose:
        print(len(train_pos), len(train_neg), len(val_pos), len(val_neg), len(test_pos), len(test_neg))
    results = evaluate_classifier(train_pos, train_neg, val_pos, val_neg, test_pos, test_neg, operators)
    return results


def print_report(exp='imdb', methods=[], selected_methods=[], threshold=0.5):
    if len(methods) == 0:
        methods = [
            'GCN', 'GAT', 'SAGE', 'GIN', 
            'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
            'GCRN', 'TIMERS', 'DynAE', 'DynRNN', 'DynAERNN', 'DynGEM', 'DySAT',
            'VGRNN', 'EvolveGCN', 'CTGCN-C',
        ]
    if len(selected_methods) == 0:
        selected_methods = [
            'GCN', 'TgGAT', 'TgSAGE', 'TgGIN',
            'GCRN', 'TIMERS', 'DynAE', 'DynRNN', 'DynAERNN', 'DynGEM', 'DySAT',
            'VGRNN', 'EvolveGCN', 'CTGCN-C']
        
    try:
        pd_edges = pd.read_csv(f'./data/{exp}/0.input/temporal_edge_list.txt', sep=' ', names=['source_id', 'target_id', 'time_id'])
    except:
        pd_edges = pd.read_csv(f'./data/{exp}/0.input/edges.csv')
        
    data = pd.read_pickle(f'./data/{exp}/0.input/data.pkl')
    all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))
    cid2cname = pd.read_pickle('/home/hoang/github/DySubG/dataset/dblp/cid2cname.pkl')
    cname2cid = {j:i for i,j in cid2cname.items() if j in all_labels}
    num_time_steps = pd_edges['time_id'].max() + 1

    operators = ['HAD', 'AVG', 'L1', 'L2']
    res = []
    for method in methods:
        print(method)
        if method in ['DynAE', 'DynRNN', 'DynAERNN']:
            filtered_data = {i:j for i,j in data.items() if j['time_id']>=1}
            results, models = eval_lp(filtered_data, cname2cid, method, num_time_steps, exp, operators, threshold=threshold)
        else:
            results, models = eval_lp(data, cname2cid, method, num_time_steps, exp, operators, threshold=threshold)
        tmp = pd.DataFrame(results)
        tmp['method'] = method
        res.append(tmp)
    df = pd.concat(res)
    df = df.reset_index().rename(columns={'index':'dataset'})
    df['best_AUC'] = df[['sigmoid', 'HAD', 'AVG', 'L1', 'L2']].max(axis=1)
    df['best_F1'] = df[['sigmoid_f1', 'HAD_f1', 'AVG_f1', 'L1_f1', 'L2_f1']].max(axis=1)

    print('Full report')
    display(df)

    print('Test report')
    display(df[df['dataset']=='test'])

    print('Selected methods')
    display(df[(df['dataset']=='test')&(df['method'].isin(selected_methods))][
        ['method', 'sigmoid', 'HAD', 'AVG', 'L1', 'L2', 'best_AUC',
         'sigmoid_f1', 'HAD_f1', 'AVG_f1', 'L1_f1', 'L2_f1', 'best_F1']])
    return df, models

def make_prediction(pred_idx, node_embedding, models):
    ranking = {}
    # 
    distances = []
    for s, ts in pred_idx.items():
        for t in ts:
            source = node_embedding[s]
            target = node_embedding[t]
            probs = sigmoid(np.dot(source, target.T))
            distances.append((s, t, probs))
        distances.sort(key=lambda tup: tup[2])
    ranking['sigmoid'] = distances                   
    #
    for operator, model in models.items():
        distances = []
        for s, ts in pred_idx.items():
            for t in ts:
                source = node_embedding[s]
                target = node_embedding[t]
                feats = np.array([get_link_score(source, target, operator)])
                probs = model.predict_proba(feats)[:, 1][0]
                distances.append((s, t, probs))
            distances.sort(key=lambda tup: tup[2])
        ranking[operator] = distances
    return ranking

def eval_ranking(pred_dict, true_dict, k):
    predicted_indices = [] # collections.OrderedDict()
    true_indices = [] # collections.OrderedDict()
    author_indices = sorted(list(true_dict.keys()))
    for aid in author_indices:
        predicted_indices.append(pred_dict[aid])
        true_indices.append(true_dict[aid])
    eval_agent = Evaluation(predicted_indices, true_indices, k)
    return eval_agent.result

In [9]:
# data = pd.read_pickle(f'./data/dblp/0.input/data.pkl')
# all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))

# new_data = {}
# for i,d in data.items():
#     pos_label = d['label']
#     neg_label = [i for i in all_labels if i not in pos_label]
#     random.shuffle(neg_label)
#     neg_label = neg_label[:len(pos_label)]
#     d['neg_label'] = neg_label
#     new_data[i] = d
    
    
# pd.to_pickle(new_data, f'./data/dblp/0.input/data.pkl')

In [5]:
methods = [
    'GCN', 'GAT', #'SAGE', 'GIN', 
    'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
    'GCRN', 'TIMERS', 'DynAE', #'DynRNN', 
    'DynAERNN', 'DynGEM', 'DySAT',
    'VGRNN', 'EvolveGCN', 'CTGCN-C',
]
df, models = print_report(exp='dblp', methods=methods)

GCN
5949 12311 1409 2786 759 1886
GAT
5949 12311 1409 2786 759 1886
TgGCN
5949 12311 1409 2786 759 1886
TgGAT
5949 12311 1409 2786 759 1886
TgSAGE
5949 12311 1409 2786 759 1886
TgGIN
5949 12311 1409 2786 759 1886
GCRN
5949 12311 1409 2786 759 1886
TIMERS
5949 12311 1409 2786 759 1886
DynAE
5193 10667 1409 2786 759 1886
DynAERNN
5193 10667 1409 2786 759 1886
DynGEM
5949 12311 1409 2786 759 1886
DySAT
5949 12311 1409 2786 759 1886
VGRNN
5949 12311 1409 2786 759 1886
EvolveGCN
5949 12311 1409 2786 759 1886
CTGCN-C
5949 12311 1409 2786 759 1886
Full report


,dataset,sigmoid,sigmoid_f1,HAD,HAD_f1,AVG,AVG_f1,L1,L1_f1,L2,L2_f1,method,best_AUC,best_F1
0,train,0.580619,0.489292,0.555100,0.000000,0.555284,0.000000,0.576915,0.000000,0.574239,0.000000,GCN,0.580619,0.489292
1,val,0.604251,0.501357,0.527273,0.000000,0.574047,0.000000,0.594976,0.000000,0.593025,0.000000,GCN,0.604251,0.501357
2,test,0.612739,0.444248,0.503518,0.000000,0.471489,0.000000,0.602248,0.000000,0.603026,0.000000,GCN,0.612739,0.444248
3,train,0.500000,0.491470,0.555153,0.000000,0.555706,0.000000,0.565627,0.000000,0.565508,0.000000,GAT,0.565627,0.491470
4,val,0.500000,0.502855,0.558240,0.000000,0.558657,0.000000,0.574856,0.000000,0.574362,0.000000,GAT,0.574856,0.502855
5,test,0.500000,0.445946,0.523163,0.000000,0.523001,0.000000,0.568420,0.000000,0.570361,0.000000,GAT,0.570361,0.445946
6,train,0.550593,0.202253,0.549053,0.000000,0.538984,0.000000,0.518486,0.000000,0.501304,0.000000,TgGCN,0.550593,0.202253
7,val,0.556282,0.214088,0.556717,0.000000,0.570001,0.000000,0.506594,0.000000,0.490669,0.000000,TgGCN,0.570001,0.214088
8,test,0.561694,0.265976,0.562762,0.000000,0.481103,0.000000,0.480071,0.000000,0.480530,0.000000,TgGCN,0.562762,0.265976
9,train,0.535343,0.475571,0.554155,0.000000,0.542727,0.000000,0.567358,0.000000,0.566798,0.000000,TgGAT,0.567358,0.475571


Test report


,dataset,sigmoid,sigmoid_f1,HAD,HAD_f1,AVG,AVG_f1,L1,L1_f1,L2,L2_f1,method,best_AUC,best_F1
2,test,0.612739,0.444248,0.503518,0.000000,0.471489,0.000000,0.602248,0.000000,0.603026,0.000000,GCN,0.612739,0.444248
5,test,0.500000,0.445946,0.523163,0.000000,0.523001,0.000000,0.568420,0.000000,0.570361,0.000000,GAT,0.570361,0.445946
8,test,0.561694,0.265976,0.562762,0.000000,0.481103,0.000000,0.480071,0.000000,0.480530,0.000000,TgGCN,0.562762,0.265976
11,test,0.540466,0.453242,0.504695,0.000000,0.501387,0.000000,0.591223,0.000000,0.591225,0.000000,TgGAT,0.591225,0.453242
14,test,0.532003,0.398821,0.477569,0.000000,0.486947,0.000000,0.566145,0.000000,0.566339,0.000000,TgSAGE,0.566339,0.398821
17,test,0.549542,0.213725,0.409650,0.000000,0.461855,0.000000,0.470594,0.000000,0.426502,0.000000,TgGIN,0.549542,0.213725
20,test,0.536459,0.437811,0.592245,0.049080,0.567810,0.000000,0.572748,0.240588,0.575493,0.012837,GCRN,0.592245,0.437811
23,test,0.684041,0.509038,0.756864,0.410697,0.513067,0.000000,0.581598,0.332040,0.580275,0.365759,TIMERS,0.756864,0.509038
26,test,0.504181,0.445946,0.458885,0.000000,0.597534,0.000000,0.552572,0.000000,0.435636,0.000000,DynAE,0.597534,0.445946
29,test,0.414933,0.445946,0.410490,0.000000,0.593267,0.000000,0.552150,0.000000,0.447785,0.000000,DynAERNN,0.593267,0.445946


Selected methods


,method,sigmoid,HAD,AVG,L1,L2,best_AUC,sigmoid_f1,HAD_f1,AVG_f1,L1_f1,L2_f1,best_F1
2,GCN,0.612739,0.503518,0.471489,0.602248,0.603026,0.612739,0.444248,0.000000,0.000000,0.000000,0.000000,0.444248
11,TgGAT,0.540466,0.504695,0.501387,0.591223,0.591225,0.591225,0.453242,0.000000,0.000000,0.000000,0.000000,0.453242
14,TgSAGE,0.532003,0.477569,0.486947,0.566145,0.566339,0.566339,0.398821,0.000000,0.000000,0.000000,0.000000,0.398821
17,TgGIN,0.549542,0.409650,0.461855,0.470594,0.426502,0.549542,0.213725,0.000000,0.000000,0.000000,0.000000,0.213725
20,GCRN,0.536459,0.592245,0.567810,0.572748,0.575493,0.592245,0.437811,0.049080,0.000000,0.240588,0.012837,0.437811
23,TIMERS,0.684041,0.756864,0.513067,0.581598,0.580275,0.756864,0.509038,0.410697,0.000000,0.332040,0.365759,0.509038
26,DynAE,0.504181,0.458885,0.597534,0.552572,0.435636,0.597534,0.445946,0.000000,0.000000,0.000000,0.000000,0.445946
29,DynAERNN,0.414933,0.410490,0.593267,0.552150,0.447785,0.593267,0.445946,0.000000,0.000000,0.000000,0.000000,0.445946
32,DynGEM,0.567981,0.545054,0.583522,0.445676,0.507668,0.583522,0.445946,0.166840,0.000000,0.168474,0.372968,0.445946
35,DySAT,0.505889,0.514286,0.514579,0.538453,0.536407,0.538453,0.436100,0.000000,0.000000,0.007853,0.010403,0.436100


In [14]:
def get_ground_truth(data, all_labels, label_mapping):
    ground_truth = {}
    pred_idx = {}
    for i,d in data.items():
        node_id = d['node_id']
        time_id = d['time_id']
        pos_labels = d['label']
        neg_labels = [i for i in all_labels if i not in pos_labels]
        if len(pos_labels) > 0:
            ground_truth[node_id] = [label_mapping[l] for l in pos_labels]
        pred_idx[node_id] = [label_mapping[l] for l in all_labels]
    return ground_truth, pred_idx

In [21]:
exp = 'dblp'
data = pd.read_pickle(f'./data/{exp}/0.input/data.pkl')
all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))
cid2cname = pd.read_pickle('/home/hoang/github/DySubG/dataset/dblp/cid2cname.pkl')
label_mapping = {j:i for i,j in cid2cname.items() if j in all_labels}
ground_truth = {}
pred_idx = {}
test_data = {i: d for i,d in data.items() if d['dataset']=='test'}
# ground_truth, pred_idx = get_ground_truth(test_data, all_labels, label_mapping)
for i,d in test_data.items():
    node_id = d['node_id']
    time_id = d['time_id']
    pos_labels = d['label']
    neg_labels = [i for i in all_labels if i not in pos_labels]
    if time_id == 6:
        if len(pos_labels) > 0:
            ground_truth[node_id] = [cname2cid[l] for l in pos_labels]
        pred_idx[node_id] = [cname2cid[l] for l in all_labels]

In [22]:
exp = 'dblp'
for method in methods:
    time_id = 6
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t').values

    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = eval_ranking(pred, ground_truth, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    print(method)
    display(pd_res[pd_res['k']==1])

GCN


,k,recall,mrr,map,ndcg,operator
1,1,0.319597,0.444234,0.319597,0.444234,sigmoid
1,1,0.243825,0.351607,0.243825,0.351607,HAD
1,1,0.168683,0.224953,0.168683,0.224953,AVG
1,1,0.228387,0.332703,0.228387,0.332703,L1
1,1,0.231695,0.330813,0.231695,0.330813,L2


GAT


,k,recall,mrr,map,ndcg,operator
1,1,0.232955,0.334594,0.232955,0.334594,sigmoid
1,1,0.232955,0.334594,0.232955,0.334594,HAD
1,1,0.319282,0.402647,0.319282,0.402647,AVG
1,1,0.174827,0.262760,0.174827,0.262760,L1
1,1,0.199401,0.281664,0.199401,0.281664,L2


TgGCN


...

,k,recall,mrr,map,ndcg,operator
1,1,0.307940,0.391304,0.307940,0.391304,sigmoid
1,1,0.155608,0.249527,0.155608,0.249527,HAD
1,1,0.146156,0.240076,0.146156,0.240076,AVG
1,1,0.146156,0.240076,0.146156,0.240076,L1
1,1,0.319282,0.402647,0.319282,0.402647,L2


EvolveGCN


,k,recall,mrr,map,ndcg,operator
1,1,0.207908,0.304348,0.207908,0.304348,sigmoid
1,1,0.325898,0.431002,0.325898,0.431002,HAD
1,1,0.232955,0.334594,0.232955,0.334594,AVG
1,1,0.232955,0.334594,0.232955,0.334594,L1
1,1,0.232955,0.334594,0.232955,0.334594,L2


CTGCN-C


,k,recall,mrr,map,ndcg,operator
1,1,0.221141,0.311909,0.221141,0.311909,sigmoid
1,1,0.226024,0.323251,0.226024,0.323251,HAD
1,1,0.168683,0.224953,0.168683,0.224953,AVG
1,1,0.214052,0.285444,0.214052,0.285444,L1
1,1,0.207908,0.283554,0.207908,0.283554,L2


In [23]:
exp = 'dblp'
for method in methods:
    time_id = 6
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t').values

    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = eval_ranking(pred, ground_truth, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    print(method)
    display(pd_res[pd_res['k']==2])

GCN


,k,recall,mrr,map,ndcg,operator
2,2,0.542943,0.551040,0.451024,0.524945,sigmoid
2,2,0.463548,0.456522,0.372180,0.440823,HAD
2,2,0.487965,0.389414,0.345164,0.403977,AVG
2,2,0.431411,0.442344,0.344140,0.414189,L1
2,2,0.460870,0.449905,0.361862,0.432844,L2


GAT


,k,recall,mrr,map,ndcg,operator
2,2,0.393604,0.408318,0.335082,0.388377,sigmoid
2,2,0.393604,0.408318,0.335082,0.388377,HAD
2,2,0.487965,0.478261,0.420463,0.463351,AVG
2,2,0.363831,0.372401,0.280813,0.346405,L1
2,2,0.404159,0.395085,0.313579,0.376425,L2


TgGCN


...

,k,recall,mrr,map,ndcg,operator
2,2,0.522779,0.524575,0.424874,0.495112,sigmoid
2,2,0.306648,0.327977,0.245841,0.307046,HAD
2,2,0.291525,0.315690,0.233554,0.294016,AVG
2,2,0.291525,0.315690,0.233554,0.294016,L1
2,2,0.539792,0.538752,0.439052,0.510032,L2


EvolveGCN


,k,recall,mrr,map,ndcg,operator
2,2,0.376118,0.382798,0.311216,0.367943,sigmoid
2,2,0.487492,0.501890,0.424401,0.478430,HAD
2,2,0.393604,0.408318,0.335082,0.388377,AVG
2,2,0.393604,0.408318,0.335082,0.388377,L1
2,2,0.393604,0.408318,0.335082,0.388377,L2


CTGCN-C


,k,recall,mrr,map,ndcg,operator
2,2,0.428733,0.414934,0.340044,0.398009,sigmoid
2,2,0.396440,0.417769,0.321692,0.383765,HAD
2,2,0.389193,0.378072,0.282624,0.349158,AVG
2,2,0.464020,0.423440,0.353986,0.414838,L1
2,2,0.420227,0.402647,0.325866,0.382547,L2


In [7]:
exp = 'dblp'
for method in methods:
    time_id = 6
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t').values

    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = eval_ranking(pred, ground_truth, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    print(method)
    display(pd_res[pd_res['k']==2])

GCN


,k,recall,mrr,map,ndcg,operator
2,2,0.547222,0.555243,0.456164,0.529393,sigmoid
2,2,0.464825,0.455993,0.372441,0.440931,HAD
2,2,0.487141,0.388577,0.344741,0.403249,AVG
2,2,0.436735,0.447566,0.350281,0.419674,L1
2,2,0.465918,0.453184,0.365964,0.436772,L2


GAT


,k,recall,mrr,map,ndcg,operator
2,2,0.383989,0.400749,0.328901,0.384407,sigmoid
2,2,0.383989,0.400749,0.328901,0.384407,HAD
2,2,0.487141,0.476592,0.419335,0.462067,AVG
2,2,0.369788,0.378277,0.287547,0.352524,L1
2,2,0.409738,0.398876,0.318134,0.380882,L2


TgGCN


...

,k,recall,mrr,map,ndcg,operator
2,2,0.519757,0.520599,0.421832,0.491657,sigmoid
2,2,0.307522,0.327715,0.246348,0.307225,HAD
2,2,0.292541,0.315543,0.234176,0.294317,AVG
2,2,0.298159,0.322097,0.240730,0.300627,L1
2,2,0.534738,0.533708,0.434941,0.505256,L2


EvolveGCN


,k,recall,mrr,map,ndcg,operator
2,2,0.394444,0.412921,0.338187,0.393881,sigmoid
2,2,0.479963,0.480337,0.411845,0.466034,HAD
2,2,0.383989,0.400749,0.328901,0.384407,AVG
2,2,0.393976,0.404494,0.331398,0.389500,L1
2,2,0.384769,0.398876,0.326717,0.383939,L2


CTGCN-C


,k,recall,mrr,map,ndcg,operator
2,2,0.450936,0.436330,0.358162,0.417010,sigmoid
2,2,0.398346,0.418539,0.323361,0.385098,HAD
2,2,0.389295,0.377341,0.282787,0.348943,AVG
2,2,0.469039,0.428839,0.360034,0.420317,L1
2,2,0.423783,0.403558,0.327497,0.384382,L2


In [15]:
methods = [
    'GCN', 'GAT', #'SAGE', 'GIN', 
    'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
    'GCRN', 'TIMERS', 'DynAE', #'DynRNN', 
    'DynAERNN', 'DynGEM', 'DySAT',
    'VGRNN', 'EvolveGCN', 'CTGCN-C',
]
df, models = print_report(exp='dblp', methods=methods)

GCN
5949 12311 1409 2786 759 1886
GAT
5949 12311 1409 2786 759 1886
TgGCN
5949 12311 1409 2786 759 1886
TgGAT
5949 12311 1409 2786 759 1886
TgSAGE
5949 12311 1409 2786 759 1886
TgGIN
5949 12311 1409 2786 759 1886
GCRN
5949 12311 1409 2786 759 1886
TIMERS
5949 12311 1409 2786 759 1886
DynAE
5193 10667 1409 2786 759 1886
DynAERNN
5193 10667 1409 2786 759 1886
DynGEM
5949 12311 1409 2786 759 1886
DySAT
5949 12311 1409 2786 759 1886
VGRNN
5949 12311 1409 2786 759 1886
EvolveGCN
5949 12311 1409 2786 759 1886
CTGCN-C
5949 12311 1409 2786 759 1886
Full report


,dataset,sigmoid,sigmoid_f1,HAD,HAD_f1,AVG,AVG_f1,L1,L1_f1,L2,L2_f1,method,best_AUC,best_F1
0,train,0.580619,0.489292,0.555100,0.000000,0.555284,0.000000,0.576915,0.000000,0.574239,0.000000,GCN,0.580619,0.489292
1,val,0.604251,0.501357,0.527273,0.000000,0.574047,0.000000,0.594976,0.000000,0.593025,0.000000,GCN,0.604251,0.501357
2,test,0.612739,0.444248,0.503518,0.000000,0.471489,0.000000,0.602248,0.000000,0.603026,0.000000,GCN,0.612739,0.444248
3,train,0.500000,0.491470,0.555153,0.000000,0.555706,0.000000,0.565627,0.000000,0.565508,0.000000,GAT,0.565627,0.491470
4,val,0.500000,0.502855,0.558240,0.000000,0.558657,0.000000,0.574856,0.000000,0.574362,0.000000,GAT,0.574856,0.502855
5,test,0.500000,0.445946,0.523163,0.000000,0.523001,0.000000,0.568420,0.000000,0.570361,0.000000,GAT,0.570361,0.445946
6,train,0.550593,0.202253,0.549053,0.000000,0.538984,0.000000,0.518486,0.000000,0.501304,0.000000,TgGCN,0.550593,0.202253
7,val,0.556282,0.214088,0.556717,0.000000,0.570001,0.000000,0.506594,0.000000,0.490669,0.000000,TgGCN,0.570001,0.214088
8,test,0.561694,0.265976,0.562762,0.000000,0.481103,0.000000,0.480071,0.000000,0.480530,0.000000,TgGCN,0.562762,0.265976
9,train,0.535343,0.475571,0.554155,0.000000,0.542727,0.000000,0.567358,0.000000,0.566798,0.000000,TgGAT,0.567358,0.475571


Test report


,dataset,sigmoid,sigmoid_f1,HAD,HAD_f1,AVG,AVG_f1,L1,L1_f1,L2,L2_f1,method,best_AUC,best_F1
2,test,0.612739,0.444248,0.503518,0.000000,0.471489,0.000000,0.602248,0.000000,0.603026,0.000000,GCN,0.612739,0.444248
5,test,0.500000,0.445946,0.523163,0.000000,0.523001,0.000000,0.568420,0.000000,0.570361,0.000000,GAT,0.570361,0.445946
8,test,0.561694,0.265976,0.562762,0.000000,0.481103,0.000000,0.480071,0.000000,0.480530,0.000000,TgGCN,0.562762,0.265976
11,test,0.540466,0.453242,0.504695,0.000000,0.501387,0.000000,0.591223,0.000000,0.591225,0.000000,TgGAT,0.591225,0.453242
14,test,0.532003,0.398821,0.477569,0.000000,0.486947,0.000000,0.566145,0.000000,0.566339,0.000000,TgSAGE,0.566339,0.398821
17,test,0.549542,0.213725,0.409650,0.000000,0.461855,0.000000,0.470594,0.000000,0.426502,0.000000,TgGIN,0.549542,0.213725
20,test,0.536459,0.437811,0.592245,0.049080,0.567810,0.000000,0.572748,0.240588,0.575493,0.012837,GCRN,0.592245,0.437811
23,test,0.684041,0.509038,0.756864,0.410697,0.513067,0.000000,0.581598,0.332040,0.580275,0.365759,TIMERS,0.756864,0.509038
26,test,0.504181,0.445946,0.458885,0.000000,0.597534,0.000000,0.552572,0.000000,0.435636,0.000000,DynAE,0.597534,0.445946
29,test,0.414933,0.445946,0.410490,0.000000,0.593267,0.000000,0.552150,0.000000,0.447785,0.000000,DynAERNN,0.593267,0.445946


Selected methods


,method,sigmoid,HAD,AVG,L1,L2,best_AUC,sigmoid_f1,HAD_f1,AVG_f1,L1_f1,L2_f1,best_F1
2,GCN,0.612739,0.503518,0.471489,0.602248,0.603026,0.612739,0.444248,0.000000,0.000000,0.000000,0.000000,0.444248
11,TgGAT,0.540466,0.504695,0.501387,0.591223,0.591225,0.591225,0.453242,0.000000,0.000000,0.000000,0.000000,0.453242
14,TgSAGE,0.532003,0.477569,0.486947,0.566145,0.566339,0.566339,0.398821,0.000000,0.000000,0.000000,0.000000,0.398821
17,TgGIN,0.549542,0.409650,0.461855,0.470594,0.426502,0.549542,0.213725,0.000000,0.000000,0.000000,0.000000,0.213725
20,GCRN,0.536459,0.592245,0.567810,0.572748,0.575493,0.592245,0.437811,0.049080,0.000000,0.240588,0.012837,0.437811
23,TIMERS,0.684041,0.756864,0.513067,0.581598,0.580275,0.756864,0.509038,0.410697,0.000000,0.332040,0.365759,0.509038
26,DynAE,0.504181,0.458885,0.597534,0.552572,0.435636,0.597534,0.445946,0.000000,0.000000,0.000000,0.000000,0.445946
29,DynAERNN,0.414933,0.410490,0.593267,0.552150,0.447785,0.593267,0.445946,0.000000,0.000000,0.000000,0.000000,0.445946
32,DynGEM,0.567981,0.545054,0.583522,0.445676,0.507668,0.583522,0.445946,0.166840,0.000000,0.168474,0.372968,0.445946
35,DySAT,0.505889,0.514286,0.514579,0.538453,0.536407,0.538453,0.436100,0.000000,0.000000,0.007853,0.010403,0.436100


In [16]:
exp = 'dblp'
data = pd.read_pickle(f'./data/{exp}/0.input/data.pkl')
all_labels = list(set(itertools.chain(*[d['label'] for i, d in data.items()])))
cid2cname = pd.read_pickle('/home/hoang/github/DySubG/dataset/dblp/cid2cname.pkl')
cname2cid = {j:i for i,j in cid2cname.items() if j in all_labels}
ground_truth = {}
pred_idx = {}
for i,d in data.items():
    node_id = d['node_id']
    time_id = d['time_id']
    pos_labels = d['label']
    neg_labels = [i for i in all_labels if i not in pos_labels]
    if time_id == 6:
        if len(pos_labels) > 0:
            ground_truth[node_id] = [cname2cid[l] for l in pos_labels]
        pred_idx[node_id] = [cname2cid[l] for l in all_labels]

In [12]:
exp = 'dblp'
for method in methods:
    time_id = 6
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t').values

    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = eval_ranking(pred, ground_truth, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    print(method)
    display(pd_res[pd_res['k']==2])

GCN


,k,recall,mrr,map,ndcg,operator
2,2,0.464512,0.516367,0.395636,0.470063,sigmoid
2,2,0.385406,0.391162,0.297981,0.367076,HAD
2,2,0.370622,0.339607,0.262370,0.326253,AVG
2,2,0.345990,0.382160,0.270226,0.339108,L1
2,2,0.375177,0.388707,0.283252,0.353009,L2


GAT


,k,recall,mrr,map,ndcg,operator
2,2,0.267512,0.323241,0.221345,0.275474,sigmoid
2,2,0.429460,0.436989,0.316189,0.399175,HAD
2,2,0.370622,0.339607,0.262370,0.326253,AVG
2,2,0.320158,0.351064,0.249482,0.309491,L1
2,2,0.328478,0.345336,0.253983,0.312775,L2


TgGCN


...

,k,recall,mrr,map,ndcg,operator
2,2,0.416367,0.456628,0.331056,0.406738,sigmoid
2,2,0.333852,0.388707,0.266885,0.333488,HAD
2,2,0.322395,0.386252,0.266612,0.328209,AVG
2,2,0.328942,0.393617,0.273977,0.335360,L1
2,2,0.426187,0.466448,0.340876,0.416558,L2


EvolveGCN


,k,recall,mrr,map,ndcg,operator
2,2,0.248691,0.279869,0.188789,0.243811,sigmoid
2,2,0.296154,0.337152,0.241435,0.296312,HAD
2,2,0.267512,0.323241,0.221345,0.275474,AVG
2,2,0.283333,0.327332,0.230892,0.285897,L1
2,2,0.269149,0.320786,0.217730,0.273584,L2


CTGCN-C


,k,recall,mrr,map,ndcg,operator
2,2,0.312930,0.337971,0.233797,0.297977,sigmoid
2,2,0.384670,0.391980,0.293221,0.361484,HAD
2,2,0.329705,0.364975,0.277196,0.325541,AVG
2,2,0.364348,0.383797,0.282351,0.345264,L1
2,2,0.303655,0.335516,0.243412,0.297130,L2


In [ ]:
0.464512	0.516367	0.395636	0.470063	
0.392908	0.402619	0.300914	0.371440
0.429460	0.468903	0.343331	0.419227	

In [18]:
exp = 'dblp'
for method in methods:
    time_id = 6
    node_embedding = pd.read_csv(
                './data/{}/2.embedding/{}/{:02d}.csv'.format(exp, method, time_id), index_col=0, sep='\t').values

    ranking = make_prediction(pred_idx, node_embedding, models)
    eval_metrics = []
    for operator, rk in ranking.items():
        pd_pred = pd.DataFrame(rk, columns=['source', 'target', 'sims']).sort_values(['sims'], ascending=False)
        pred = pd_pred.groupby('source').agg({'target': list}).to_dict()['target']
        res = eval_ranking(pred, ground_truth, k=5)
        res['operator'] = operator
        eval_metrics.append(res)
    pd_res = pd.concat(eval_metrics)
    print(method)
    display(pd_res[pd_res['k']==3])

GCN


,k,recall,mrr,map,ndcg,operator
3,3,0.718872,0.600819,0.527447,0.599876,sigmoid
3,3,0.676181,0.526465,0.454537,0.535357,HAD
3,3,0.633333,0.451796,0.399795,0.479036,AVG
3,3,0.631285,0.499685,0.428019,0.503903,L1
3,3,0.700284,0.524890,0.458134,0.542412,L2


GAT


,k,recall,mrr,map,ndcg,operator
3,3,0.546692,0.451166,0.404521,0.458154,sigmoid
3,3,0.546692,0.451166,0.404521,0.458154,HAD
3,3,0.633333,0.540643,0.475095,0.536904,AVG
3,3,0.581191,0.438563,0.367948,0.446721,L1
3,3,0.579616,0.453686,0.385486,0.459106,L2


TgGCN


...

,k,recall,mrr,map,ndcg,operator
3,3,0.691462,0.570573,0.494786,0.569538,sigmoid
3,3,0.473440,0.390989,0.306511,0.375988,HAD
3,3,0.460208,0.379332,0.294854,0.363903,AVG
3,3,0.460208,0.379332,0.294854,0.363903,L1
3,3,0.708475,0.584751,0.508963,0.584458,L2


EvolveGCN


,k,recall,mrr,map,ndcg,operator
3,3,0.536452,0.430057,0.382493,0.439561,sigmoid
3,3,0.656490,0.556081,0.495888,0.557053,HAD
3,3,0.546692,0.451166,0.404521,0.458154,AVG
3,3,0.546692,0.451166,0.404521,0.458154,L1
3,3,0.546692,0.451166,0.404521,0.458154,L2


CTGCN-C


,k,recall,mrr,map,ndcg,operator
3,3,0.614272,0.476686,0.415312,0.487062,sigmoid
3,3,0.625299,0.487713,0.414498,0.492136,HAD
3,3,0.708475,0.471960,0.406622,0.506061,AVG
3,3,0.627190,0.477001,0.420537,0.493974,L1
3,3,0.597574,0.463138,0.393783,0.468598,L2


In [8]:
# dysat_embedding = pd.read_pickle('../DySAT_pytorch/dblp.pkl').cpu().detach().numpy()
# for i in range(8):
#     embedding_path = './data/dblp/2.embedding/DySAT/{:02d}.csv'.format(i)
#     embs = dysat_embedding[:, i, :]
#     df_export = pd.DataFrame(data=embs)
#     df_export.to_csv(embedding_path, sep='\t', header=True, index=True)

In [9]:
methods = [
    'GCN', 'GAT', #'SAGE', 'GIN', 
    'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
    'GCRN', 'TIMERS', 'DynAE', #'DynRNN', 
    'DynAERNN', 'DynGEM', 
    'VGRNN', 'EvolveGCN', 'CTGCN-C',
]
df = print_report(exp='dblp', methods=methods, threshold=0.1)

GCN
6766 16778 1634 3880 939 2691
GAT
6766 16778 1634 3880 939 2691
TgGCN
6766 16778 1634 3880 939 2691
TgGAT
6766 16778 1634 3880 939 2691
TgSAGE
6766 16778 1634 3880 939 2691
TgGIN
6766 16778 1634 3880 939 2691
GCRN
6766 16778 1634 3880 939 2691
TIMERS
6766 16778 1634 3880 939 2691
DynAE
5926 14534 1634 3880 939 2691
DynAERNN
5926 14534 1634 3880 939 2691
DynGEM
6766 16778 1634 3880 939 2691
VGRNN
6766 16778 1634 3880 939 2691
EvolveGCN
6766 16778 1634 3880 939 2691
CTGCN-C
6766 16778 1634 3880 939 2691
Full report


,dataset,sigmoid,sigmoid_f1,HAD,HAD_f1,AVG,AVG_f1,L1,L1_f1,L2,L2_f1,method,best_AUC,best_F1
0,train,0.574367,0.446195,0.566038,0.000000,0.558710,0.000000,0.572380,0.000000,0.570376,0.000000,GCN,0.574367,0.446195
1,val,0.599924,0.456618,0.545354,0.000000,0.576097,0.000000,0.590480,0.000000,0.588157,0.000000,GCN,0.599924,0.456618
2,test,0.601287,0.409750,0.528716,0.000000,0.493888,0.000000,0.606964,0.000000,0.607368,0.000000,GCN,0.607368,0.409750
3,train,0.500000,0.446453,0.551076,0.000000,0.551232,0.000000,0.568374,0.000000,0.568344,0.000000,GAT,0.568374,0.446453
4,val,0.500000,0.457191,0.559883,0.000000,0.559820,0.000000,0.572180,0.000000,0.571434,0.000000,GAT,0.572180,0.457191
5,test,0.500000,0.411031,0.536241,0.000000,0.536097,0.000000,0.574326,0.000000,0.576399,0.000000,GAT,0.576399,0.411031
6,train,0.539605,0.203712,0.538869,0.000000,0.561354,0.000000,0.528697,0.000000,0.485712,0.000000,TgGCN,0.561354,0.203712
7,val,0.552159,0.205827,0.551792,0.000000,0.573810,0.000000,0.512906,0.000000,0.484653,0.000000,TgGCN,0.573810,0.205827
8,test,0.577050,0.258741,0.577725,0.000000,0.484433,0.000000,0.487109,0.000000,0.513177,0.000000,TgGCN,0.577725,0.258741
9,train,0.528802,0.434968,0.526655,0.000000,0.550534,0.000000,0.562202,0.000000,0.561647,0.000000,TgGAT,0.562202,0.434968


Test report


,dataset,sigmoid,sigmoid_f1,HAD,HAD_f1,AVG,AVG_f1,L1,L1_f1,L2,L2_f1,method,best_AUC,best_F1
2,test,0.601287,0.409750,0.528716,0.000000,0.493888,0.000000,0.606964,0.000000,0.607368,0.000000,GCN,0.607368,0.409750
5,test,0.500000,0.411031,0.536241,0.000000,0.536097,0.000000,0.574326,0.000000,0.576399,0.000000,GAT,0.576399,0.411031
8,test,0.577050,0.258741,0.577725,0.000000,0.484433,0.000000,0.487109,0.000000,0.513177,0.000000,TgGCN,0.577725,0.258741
11,test,0.538524,0.416469,0.451944,0.000000,0.490131,0.000000,0.589372,0.000000,0.590014,0.000000,TgGAT,0.590014,0.416469
14,test,0.532665,0.373384,0.464341,0.000000,0.483685,0.000000,0.573829,0.000000,0.574548,0.000000,TgSAGE,0.574548,0.373384
17,test,0.565183,0.209192,0.429162,0.000000,0.504920,0.000000,0.466866,0.000000,0.427028,0.000000,TgGIN,0.565183,0.209192
20,test,0.533966,0.402253,0.592604,0.002123,0.559301,0.000000,0.566614,0.149466,0.568817,0.000000,GCRN,0.592604,0.402253
23,test,0.680806,0.477099,0.739314,0.312236,0.513659,0.000000,0.557099,0.135158,0.545070,0.002128,TIMERS,0.739314,0.477099
26,test,0.504422,0.411031,0.487809,0.000000,0.549287,0.000000,0.541968,0.000000,0.542958,0.000000,DynAE,0.549287,0.411031
29,test,0.443525,0.411031,0.441421,0.000000,0.563988,0.000000,0.521447,0.000000,0.478693,0.000000,DynAERNN,0.563988,0.411031


Selected methods


,method,sigmoid,HAD,AVG,L1,L2,best_AUC,sigmoid_f1,HAD_f1,AVG_f1,L1_f1,L2_f1,best_F1
2,GCN,0.601287,0.528716,0.493888,0.606964,0.607368,0.607368,0.409750,0.000000,0.000000,0.000000,0.000000,0.409750
11,TgGAT,0.538524,0.451944,0.490131,0.589372,0.590014,0.590014,0.416469,0.000000,0.000000,0.000000,0.000000,0.416469
14,TgSAGE,0.532665,0.464341,0.483685,0.573829,0.574548,0.574548,0.373384,0.000000,0.000000,0.000000,0.000000,0.373384
17,TgGIN,0.565183,0.429162,0.504920,0.466866,0.427028,0.565183,0.209192,0.000000,0.000000,0.000000,0.000000,0.209192
20,GCRN,0.533966,0.592604,0.559301,0.566614,0.568817,0.592604,0.402253,0.002123,0.000000,0.149466,0.000000,0.402253
23,TIMERS,0.680806,0.739314,0.513659,0.557099,0.545070,0.739314,0.477099,0.312236,0.000000,0.135158,0.002128,0.477099
26,DynAE,0.504422,0.487809,0.549287,0.541968,0.542958,0.549287,0.411031,0.000000,0.000000,0.000000,0.000000,0.411031
29,DynAERNN,0.443525,0.441421,0.563988,0.521447,0.478693,0.563988,0.411031,0.000000,0.000000,0.000000,0.000000,0.411031
32,DynGEM,0.574835,0.533583,0.545538,0.500516,0.486040,0.574835,0.411031,0.115419,0.002128,0.022587,0.277726,0.411031
35,VGRNN,0.567795,0.579317,0.573978,0.555672,0.553506,0.579317,0.413342,0.000000,0.000000,0.000000,0.000000,0.413342


In [18]:
methods = [
    'GCN', 'GAT', #'SAGE', 'GIN', 
    'TgGCN', 'TgGAT', 'TgSAGE', 'TgGIN', 
    'GCRN', 'TIMERS', 'DynAE', #'DynRNN', 
    'DynAERNN', 'DynGEM', 
    'VGRNN', 'EvolveGCN', 'CTGCN-C',
]
df = print_report(exp='dblp', methods=methods)

GCN
6766 16778 1634 3880 939 2691
GAT
6766 16778 1634 3880 939 2691
TgGCN
6766 16778 1634 3880 939 2691
TgGAT
6766 16778 1634 3880 939 2691
TgSAGE
6766 16778 1634 3880 939 2691
TgGIN
6766 16778 1634 3880 939 2691
GCRN
6766 16778 1634 3880 939 2691
TIMERS
6766 16778 1634 3880 939 2691
DynAE
5926 14534 1634 3880 939 2691
DynAERNN
5926 14534 1634 3880 939 2691
DynGEM
6766 16778 1634 3880 939 2691
VGRNN
6766 16778 1634 3880 939 2691
EvolveGCN
6766 16778 1634 3880 939 2691
CTGCN-C
6766 16778 1634 3880 939 2691
Full report


,dataset,sigmoid,sigmoid_f1,HAD,HAD_f1,AVG,AVG_f1,L1,L1_f1,L2,L2_f1,method,best_AUC,best_F1
0,train,0.574367,0.446195,0.566038,0.000000,0.558710,0.000000,0.572380,0.000000,0.570376,0.000000,GCN,0.574367,0.446195
1,val,0.599924,0.456618,0.545354,0.000000,0.576097,0.000000,0.590480,0.000000,0.588157,0.000000,GCN,0.599924,0.456618
2,test,0.601287,0.409750,0.528716,0.000000,0.493888,0.000000,0.606964,0.000000,0.607368,0.000000,GCN,0.607368,0.409750
3,train,0.500000,0.446453,0.551076,0.000000,0.551232,0.000000,0.568374,0.000000,0.568344,0.000000,GAT,0.568374,0.446453
4,val,0.500000,0.457191,0.559883,0.000000,0.559820,0.000000,0.572180,0.000000,0.571434,0.000000,GAT,0.572180,0.457191
5,test,0.500000,0.411031,0.536241,0.000000,0.536097,0.000000,0.574326,0.000000,0.576399,0.000000,GAT,0.576399,0.411031
6,train,0.539605,0.203712,0.538869,0.000000,0.561354,0.000000,0.528697,0.000000,0.485712,0.000000,TgGCN,0.561354,0.203712
7,val,0.552159,0.205827,0.551792,0.000000,0.573810,0.000000,0.512906,0.000000,0.484653,0.000000,TgGCN,0.573810,0.205827
8,test,0.577050,0.258741,0.577725,0.000000,0.484433,0.000000,0.487109,0.000000,0.513177,0.000000,TgGCN,0.577725,0.258741
9,train,0.528802,0.434968,0.526655,0.000000,0.550534,0.000000,0.562202,0.000000,0.561647,0.000000,TgGAT,0.562202,0.434968


Test report


,dataset,sigmoid,sigmoid_f1,HAD,HAD_f1,AVG,AVG_f1,L1,L1_f1,L2,L2_f1,method,best_AUC,best_F1
2,test,0.601287,0.409750,0.528716,0.000000,0.493888,0.000000,0.606964,0.000000,0.607368,0.000000,GCN,0.607368,0.409750
5,test,0.500000,0.411031,0.536241,0.000000,0.536097,0.000000,0.574326,0.000000,0.576399,0.000000,GAT,0.576399,0.411031
8,test,0.577050,0.258741,0.577725,0.000000,0.484433,0.000000,0.487109,0.000000,0.513177,0.000000,TgGCN,0.577725,0.258741
11,test,0.538524,0.416469,0.451944,0.000000,0.490131,0.000000,0.589372,0.000000,0.590014,0.000000,TgGAT,0.590014,0.416469
14,test,0.532665,0.373384,0.464341,0.000000,0.483685,0.000000,0.573829,0.000000,0.574548,0.000000,TgSAGE,0.574548,0.373384
17,test,0.565183,0.209192,0.429162,0.000000,0.504920,0.000000,0.466866,0.000000,0.427028,0.000000,TgGIN,0.565183,0.209192
20,test,0.533966,0.402253,0.592604,0.002123,0.559301,0.000000,0.566614,0.149466,0.568817,0.000000,GCRN,0.592604,0.402253
23,test,0.680806,0.477099,0.739314,0.312236,0.513659,0.000000,0.557099,0.135158,0.545070,0.002128,TIMERS,0.739314,0.477099
26,test,0.504422,0.411031,0.487809,0.000000,0.549287,0.000000,0.541968,0.000000,0.542958,0.000000,DynAE,0.549287,0.411031
29,test,0.443525,0.411031,0.441421,0.000000,0.563988,0.000000,0.521447,0.000000,0.478693,0.000000,DynAERNN,0.563988,0.411031


Selected methods


,method,sigmoid,HAD,AVG,L1,L2,best_AUC,sigmoid_f1,HAD_f1,AVG_f1,L1_f1,L2_f1,best_F1
2,GCN,0.601287,0.528716,0.493888,0.606964,0.607368,0.607368,0.409750,0.000000,0.000000,0.000000,0.000000,0.409750
11,TgGAT,0.538524,0.451944,0.490131,0.589372,0.590014,0.590014,0.416469,0.000000,0.000000,0.000000,0.000000,0.416469
14,TgSAGE,0.532665,0.464341,0.483685,0.573829,0.574548,0.574548,0.373384,0.000000,0.000000,0.000000,0.000000,0.373384
17,TgGIN,0.565183,0.429162,0.504920,0.466866,0.427028,0.565183,0.209192,0.000000,0.000000,0.000000,0.000000,0.209192
20,GCRN,0.533966,0.592604,0.559301,0.566614,0.568817,0.592604,0.402253,0.002123,0.000000,0.149466,0.000000,0.402253
23,TIMERS,0.680806,0.739314,0.513659,0.557099,0.545070,0.739314,0.477099,0.312236,0.000000,0.135158,0.002128,0.477099
26,DynAE,0.504422,0.487809,0.549287,0.541968,0.542958,0.549287,0.411031,0.000000,0.000000,0.000000,0.000000,0.411031
29,DynAERNN,0.443525,0.441421,0.563988,0.521447,0.478693,0.563988,0.411031,0.000000,0.000000,0.000000,0.000000,0.411031
32,DynGEM,0.574835,0.533583,0.545538,0.500516,0.486040,0.574835,0.411031,0.115419,0.002128,0.022587,0.277726,0.411031
35,VGRNN,0.567795,0.579317,0.573978,0.555672,0.553506,0.579317,0.413342,0.000000,0.000000,0.000000,0.000000,0.413342


In [5]:
a.shape

torch.Size([11512, 5, 128])